In [ ]:
# -*- coding: utf-8 -*-

from websocket import create_connection
import gzip
import time
import pandas as pd
import json
import base64
import datetime
import hashlib
import hmac
import json
import urllib
import urllib.parse
import urllib.request
import requests
import matplotlib.pyplot as plt
import numpy as np
from HuobiDMService import HuobiDM
from pprint import pprint
import os
import abupy
import datetime

ACCESS_KEY = "*******************"
SECRET_KEY = "*****************"


# API 请求地址
URL="https://api.hbdm.com"
dm = HuobiDM(URL, ACCESS_KEY, SECRET_KEY)

dfTime=0 #打印dataframe的时间
buytag=0 #0没买  1买了
buyTime=0
selltag=0 #0没卖  1买了
sellTime=0
lastflag=0
lowPriceTag=99999
topPriceTag=0

def getDataFrame(symbolstr):  
    resp=dm.get_contract_kline(symbol=symbolstr, period='15min', size=400)
    r2=json.dumps(resp)
    loaded_r2 = json.loads(r2)
    df=pd.DataFrame(loaded_r2['data'])
    df=df.sort_values(by=['id']).reset_index(drop=True)
    eos_sma20=df['close'].rolling(15).mean()
    df['sma20']=eos_sma20
    df['tmp']=df['close'].rolling(15).std()
    df['boll-top']=df['sma20']+2*df['tmp']
    df['boll-low']=df['sma20']-2*df['tmp']
    #plt.plot(df['close'],'y')
    #plt.plot(df['sma20'],'b--')
    #plt.plot(df['boll-top'],'r')
    #plt.plot(df['boll-low'],'r')
    #plt.legend()
    #plt.show()
    return df

def returnflag(eosdf,present_price):
    top=eosdf.iloc[-1]['boll-top']
    low=eosdf.iloc[-1]['boll-low']
    #print('low: ',low,'     top: ',top)
    if(low<=present_price<top):
        return 0 #布林带中间
    elif(present_price<low):
        return -1 #布林带下方
    else:
        return 1 #布林带上方

def openbid(price,amount):
    return dm.send_contract_order(symbol='EOS', contract_type='quarter', contract_code='', 
                        client_order_id='', price=price, volume=amount, direction='sell',
                        offset='open', lever_rate=20, order_price_type='limit')

def closebid(price,amount):
    return dm.send_contract_order(symbol='EOS', contract_type='quarter', contract_code='', 
                        client_order_id='', price=price, volume=amount, direction='sell',
                        offset='close', lever_rate=20, order_price_type='limit')

def openask(price,amount):
    return dm.send_contract_order(symbol='EOS', contract_type='quarter', contract_code='', 
                        client_order_id='', price=price, volume=amount, direction='buy',
                        offset='open', lever_rate=20, order_price_type='limit')

def closeask(price,amount):
    return dm.send_contract_order(symbol='EOS', contract_type='quarter', contract_code='', 
                        client_order_id='', price=price, volume=amount, direction='buy',
                        offset='close', lever_rate=20, order_price_type='limit')


if __name__ == '__main__':
    while(1):
        try:
            ws = create_connection("wss://www.hbdm.com/ws")
            break
        except:
            print('connect ws error,retry...')
            time.sleep(5)

    # 订阅 KLine 数据
    
    tradeStr_kline="""
    {"sub": "market.BTC_CQ.kline.1min",  "id": "id1"}
    """

    # 订阅 Market Detail 数据
    tradeStr_marketDetail="""
    {"sub": "market.BTC_CQ.detail",  "id": "id6" }
    """

    # 订阅 Trade Detail 数据
    tradeStr_tradeDetail="""
    {"sub": "market.BTC_CQ.trade.detail", "id": "id7"}
    """

    # 请求 KLine 数据
    tradeStr_klinereq="""
    {"req": "market.BTC_CQ.kline.1min", "id": "id4"}
    """

    # 请求 Trade Detail 数据
    tradeStr_tradeDetail_req="""
    {"req": "market.EOS_CQ.trade.detail", "id": "id5"}
    """

    # 订阅 Market Depth 数据
    tradeStr_marketDepth="""
    {
        "sub": "market.EOS_CQ.depth.step0", "id": "id9"
    }
    """
    pricedf=getDataFrame("EOS_CQ")
    dfTime=time.time()
    ws.send(tradeStr_marketDepth)
    trade_id = ''
    while(1):
        try:
            compressData=ws.recv()
            result=gzip.decompress(compressData).decode('utf-8')
            if result[:7] == '{"ping"':
                ts=result[8:21]
                pong='{"pong":'+ts+'}'
                ws.send(pong)
                ws.send(tradeStr_marketDepth)
            else:
                try:
                    if trade_id == result['data']['id']:
                        print('重复的id')
                        break
                    else:
                        trade_id = result['data']['id']
                except Exception:
                    pass
                try:
                    if time.time()-dfTime>899:
                        pricedf=getDataFrame("EOS_CQ")
                    data_json=(json.loads(result))
                    bidprice=data_json['tick']['bids'][0][0]
                    askprice=data_json['tick']['asks'][0][0]
                    flag=returnflag(pricedf,bidprice)
                    if flag==0 and lastflag==0:
                        lastflag=flag
                    elif lastflag==-1 and flag==-1: # 布林带下方徘徊
                        if bidprice<lowPriceTag:
                            lowPriceTag=bidprice
                        lastflag=flag
                    elif lastflag==1 and flag==1:
                        if bidprice>topPriceTag:
                            topPriceTag=bidprice
                        lastflag=flag
                    elif lastflag==-1 and flag==0: #从下至上突破布林带下限
                        if bidprice>lowPriceTag and (time.time()-buyTime)>600:
                            print ('平空开多','  时间: ',datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),' 价格: ',askprice)
                            closeask(askprice,1)
                            openask(bidprice,1)                        
                            buyTime=time.time()
                        lastflag=flag
                    elif lastflag==1 and flag==0: #从上至下突破布林带上限
                        if bidprice<topPriceTag and (time.time()-buyTime)>600:
                            print('平多开空','  时间: ',datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'),' 价格: ',askprice)
                            closebid(bidprice,1)
                            openbid(askprice,1)
                            buyTime=time.time()
                        lastflag=flag
                    lastflag=flag
                except Exception:
                    pass
        except Exception:
                pass

平多开空   时间:  2019-06-13 09:40:08  价格:  6.485
平多开空   时间:  2019-06-13 10:15:00  价格:  6.623
平空开多   时间:  2019-06-13 15:22:38  价格:  6.558
平多开空   时间:  2019-06-13 16:50:38  价格:  6.609
平多开空   时间:  2019-06-13 17:40:15  价格:  6.604
平空开多   时间:  2019-06-13 19:19:55  价格:  6.546
平空开多   时间:  2019-06-13 19:30:00  价格:  6.532
平多开空   时间:  2019-06-13 21:36:04  价格:  6.6
平空开多   时间:  2019-06-13 21:46:36  价格:  6.523
平空开多   时间:  2019-06-13 22:00:31  价格:  6.507
平多开空   时间:  2019-06-13 22:27:48  价格:  6.593
